In [1]:
from Sky_Under import *
import pickle
import pandas as pd
import numpy as np
import time
from New_CSV2 import *

No final da época 19/20, como houve exceções temos de criar funções excecionais para obter os jogos de cada ano
No final da época 19/20, como houve exceções temos de criar funções excecionais para obter os jogos de cada ano


In [3]:
add_odds('Premier_League',season=2020)

C:\Users\joaom\Documents\Projetos\PYTHON\Apostas\Premier_League___\PL19_20 old
C:\Users\joaom\Documents\Projetos\PYTHON\Apostas\NewPremier_League___\NewPL19_20 new
1o arg é o NewFile
['Bournemouth', 'Newcastle United', 'Wolverhampton Wanderers', 'Tottenham Hotspur', 'Southampton', 'West Ham United', 'Manchester City', 'Brighton and Hove Albion', 'Arsenal', 'Manchester United', 'Everton', 'Burnley', 'Aston Villa', 'Chelsea', 'Sheffield United', 'Watford', 'Norwich City', 'Leicester City', 'Liverpool', 'Crystal Palace']
----------------------------------
----------------------------------
Wolves? (write "passa" if not sure Or "reset" to restart) Wolverhampton Wanderers
----------------------------------
Man City? (write "passa" if not sure Or "reset" to restart) Manchester City
----------------------------------
Man United? (write "passa" if not sure Or "reset" to restart) Manchester United
----------------------------------
Newcastle? (write "passa" if not sure Or "reset" to restart) Ne

In [8]:
def update_team_names(sky,under,league):
# vamos mudar o nome das dfs de under para o nome das dfs do sky
#assim podemos fazer um merge usando o np.where
    same = file_teams(sky,under,league)
    print(same)
    same_sky = [same[i][0] for i in range(len(same))]
    same_under = [same[i][1] for i in range(len(same))]
    #HOME
    for i in range(len(same_under)):
        
        try:
            replaceH=list(np.where(under['HT']==same_under[i])[0])
            home='HT'
        except:
            replaceH=list(np.where(under['HomeTeam']==same_under[i])[0])
            home='HomeTeam'
        
        for n in replaceH:
            under.at[n,home]=same_sky[i]
    #AWAY
    for i in range(len(same_under)):
        try:
            replaceH=list(np.where(under['AT']==same_under[i])[0])
            away='AT'
        except:
            replaceH=list(np.where(under['AwayTeam']==same_under[i])[0])
            away='AwayTeam'
        for n in replaceH:
            under.at[n,away]=same_sky[i]  
    return sky,under

In [10]:
def file_teams(w1,w2,league): #w1 -> df Sky |||| w2 -> df Under
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//Teams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome Under]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        under_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        try:
            sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
            under_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
        except:
            try:
                sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
                under_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
            except:
                sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
                under_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    i = 0
    under_teams_copy = under_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(under_teams) and not(found):
                if sky_teams[ii]==under_teams[nn]:
                    same+=[[sky_teams[ii],under_teams[nn]]]
                    del under_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            under_teams_copy=under_teams.copy()
            same_copy=same.copy()
            t=0
            while under_teams!=[]:
                print('------')
                print(restos)
                ok=input(str(under_teams[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    under_teams=under_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,under_teams[t]]]
                    del under_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(under_teams):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_under = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_under = [under_teams[i] for i in range(len(under_teams)) if (under_teams[i] not in same_under)]
        missing_sky_copy = missing_sky.copy()
        missing_under_copy = missing_under.copy()
        same_copy=same.copy()
        
        restos=[]
        if (len(missing_sky)==0) and (len(missing_under)>0):
            t=0
            print(sky_teams)
            time.sleep(0.3)
            while missing_under!=[]:
                ok = input(str(missing_under[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in sky_teams) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    missing_under=missing_under_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_under[t]]]
                    del missing_under[t]
                if t==len(missing_under):
                    t=0
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
                
                
        elif len(missing_sky)==len(missing_under):
            for ii in range(len(missing_sky)):
                nn=0
                found=False
                while nn<len(missing_under) and not(found):
                    if missing_sky[ii]==missing_under[nn]:
                        same+=[[missing_sky[ii],missing_under[nn]]]
                        del missing_under[nn]
                        found=True
                    nn+=1

                if not found:
                    restos+=[missing_sky[ii]]
        else:
            raise Exception('file_teams error. É preciso resolver erro ')
            

        if restos==[]:
            print('What')
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_under!=[]):
                if len(restos) != len(missing_under):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input(str(missing_under[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_under=missing_under_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_under[t]]]
                    del missing_under[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_under):
                    t=0
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same

In [11]:
file_teams(pd1,pd2,'Bundesliga')

['FC Augsburg', 'Fortuna Dusseldorf', 'Wolfsburg', 'Werder Bremen', 'Eintracht Frankfurt', 'Hoffenheim', 'Schalke', 'RB Leipzig', 'Cologne', "M'gladbach", '1. FC Union Berlin', 'Hertha Berlin', 'SC Paderborn 07', 'Mainz', 'Bayern Munich', 'SC Freiburg', 'Borussia Dortmund', 'Bayer Leverkusen']
Ein Frankfurt? (write "passa" if not sure Or "reset" to restart) Eintracht Frankfurt
Fortuna Dusseldorf? (write "passa" if not sure Or "reset" to restart) Fortuna Dusseldorf
RB Leipzig? (write "passa" if not sure Or "reset" to restart) RB Leipzig
FC Koln? (write "passa" if not sure Or "reset" to restart) Cologne
M'gladbach? (write "passa" if not sure Or "reset" to restart) M'gladbach
Leverkusen? (write "passa" if not sure Or "reset" to restart) Bayer Leverkusen
Hertha? (write "passa" if not sure Or "reset" to restart) Hertha Berlin
Mainz? (write "passa" if not sure Or "reset" to restart) Mainz
Dortmund? (write "passa" if not sure Or "reset" to restart) Borussia Dortmund


[['Bayer Leverkusen', 'Bayer Leverkusen'],
 ['Bayern Munich', 'Bayern Munich'],
 ['Werder Bremen', 'Werder Bremen'],
 ['Hoffenheim', 'Hoffenheim'],
 ['Borussia Dortmund', 'Borussia Dortmund'],
 ['Wolfsburg', 'Wolfsburg'],
 ['Hertha Berlin', 'Hertha Berlin'],
 ['Eintracht Frankfurt', 'Eintracht Frankfurt'],
 ['Schalke', 'Schalke 04'],
 ["M'gladbach", 'Borussia M.Gladbach'],
 ['SC Freiburg', 'Freiburg'],
 ['Mainz', 'Mainz 05'],
 ['1. FC Union Berlin', 'Union Berlin'],
 ['RB Leipzig', 'RasenBallsport Leipzig'],
 ['Fortuna Dusseldorf', 'Fortuna Duesseldorf'],
 ['SC Paderborn 07', 'Paderborn'],
 ['Cologne', 'FC Cologne'],
 ['FC Augsburg', 'Augsburg'],
 ['Eintracht Frankfurt', 'Ein Frankfurt'],
 ['Fortuna Dusseldorf', 'Fortuna Dusseldorf'],
 ['RB Leipzig', 'RB Leipzig'],
 ['Cologne', 'FC Koln'],
 ["M'gladbach", "M'gladbach"],
 ['Bayer Leverkusen', 'Leverkusen'],
 ['Hertha Berlin', 'Hertha'],
 ['Mainz', 'Mainz'],
 ['Borussia Dortmund', 'Dortmund']]

In [7]:
pd1 = pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//NewBundesliga___//NewBund19_20.xlsx')
pd2 = pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//Bundesliga___//Bund19_20.xlsx')

In [12]:
pd1_,pd2_=update_team_names(pd1,pd2,'Bundesliga')

In [13]:
np.unique(pd1_['HT'])

array(['1. FC Union Berlin', 'Bayer Leverkusen', 'Bayern Munich',
       'Borussia Dortmund', 'Cologne', 'Eintracht Frankfurt',
       'FC Augsburg', 'Fortuna Dusseldorf', 'Hertha Berlin', 'Hoffenheim',
       "M'gladbach", 'Mainz', 'RB Leipzig', 'SC Freiburg',
       'SC Paderborn 07', 'Schalke', 'Werder Bremen', 'Wolfsburg'],
      dtype=object)

In [14]:
np.unique(pd2_['HomeTeam'])

array(['1. FC Union Berlin', 'Bayer Leverkusen', 'Bayern Munich',
       'Borussia Dortmund', 'Cologne', 'Eintracht Frankfurt',
       'FC Augsburg', 'Fortuna Dusseldorf', 'Hertha Berlin', 'Hoffenheim',
       "M'gladbach", 'Mainz', 'RB Leipzig', 'SC Freiburg',
       'SC Paderborn 07', 'Schalke', 'Werder Bremen', 'Wolfsburg'],
      dtype=object)

In [2]:
add_odds('Bundesliga',season=2020)

C:\Users\joaom\Documents\Projetos\PYTHON\Apostas\Bundesliga___\Bund19_20 old
C:\Users\joaom\Documents\Projetos\PYTHON\Apostas\NewBundesliga___\NewBund19_20 new
1o arg é o NewFile
0
Bayern Munich Bayern Munich Hertha Berlin Hertha
Bayern Munich Dortmund Hertha Berlin FC Augsburg
Bayern Munich SC Freiburg Hertha Berlin Mainz
Bayern Munich Leverkusen Hertha Berlin SC Paderborn 07
Bayern Munich Werder Bremen Hertha Berlin Fortuna Dusseldorf
Bayern Munich Wolfsburg Hertha Berlin FC Koln
Bayern Munich M'gladbach Hertha Berlin Schalke
Bayern Munich Ein Frankfurt Hertha Berlin Hoffenheim
Bayern Munich 1. FC Union Berlin Hertha Berlin RB Leipzig
Bayern Munich FC Koln Hertha Berlin Dortmund
Bayern Munich FC Augsburg Hertha Berlin 1. FC Union Berlin
Bayern Munich Fortuna Dusseldorf Hertha Berlin Leverkusen
Bayern Munich Hoffenheim Hertha Berlin Werder Bremen
Bayern Munich Mainz Hertha Berlin M'gladbach
Bayern Munich SC Paderborn 07 Hertha Berlin SC Freiburg
Bayern Munich Schalke Hertha Berlin Bay

Bayern Munich Hertha Hertha Berlin FC Koln
Bayern Munich M'gladbach Hertha Berlin Hoffenheim
Bayern Munich Werder Bremen Hertha Berlin Dortmund
Bayern Munich Schalke Hertha Berlin RB Leipzig
Bayern Munich Leverkusen Hertha Berlin FC Augsburg
Bayern Munich Wolfsburg Hertha Berlin Mainz
Bayern Munich Ein Frankfurt Hertha Berlin 1. FC Union Berlin
Bayern Munich Fortuna Dusseldorf Hertha Berlin Hertha
Bayern Munich FC Augsburg Hertha Berlin M'gladbach
Bayern Munich Dortmund Hertha Berlin SC Freiburg
Bayern Munich Hoffenheim Hertha Berlin Bayern Munich
Bayern Munich Mainz Hertha Berlin SC Paderborn 07
Bayern Munich FC Koln Hertha Berlin Schalke
Bayern Munich 1. FC Union Berlin Hertha Berlin Wolfsburg
Bayern Munich RB Leipzig Hertha Berlin Leverkusen
Bayern Munich SC Paderborn 07 Hertha Berlin FC Koln
Bayern Munich SC Freiburg Hertha Berlin 1. FC Union Berlin
Bayern Munich Hertha Hertha Berlin Werder Bremen
Bayern Munich Leverkusen Hertha Berlin Ein Frankfurt
Bayern Munich Schalke Hertha Ber

Borussia Dortmund Wolfsburg FC Augsburg Leverkusen
Borussia Dortmund SC Freiburg FC Augsburg Ein Frankfurt
Borussia Dortmund Dortmund FC Augsburg SC Paderborn 07
Borussia Dortmund Ein Frankfurt FC Augsburg Wolfsburg
Borussia Dortmund Fortuna Dusseldorf FC Augsburg Bayern Munich
Borussia Dortmund Leverkusen FC Augsburg SC Freiburg
Borussia Dortmund 1. FC Union Berlin FC Augsburg M'gladbach
Borussia Dortmund Werder Bremen FC Augsburg Schalke
Borussia Dortmund RB Leipzig FC Augsburg FC Koln
Borussia Dortmund FC Augsburg FC Augsburg Hertha
Borussia Dortmund Hoffenheim FC Augsburg Mainz
Borussia Dortmund Schalke FC Augsburg 1. FC Union Berlin
Borussia Dortmund FC Koln FC Augsburg FC Augsburg
Borussia Dortmund Hertha FC Augsburg Dortmund
Borussia Dortmund Hoffenheim FC Augsburg Fortuna Dusseldorf
Borussia Dortmund SC Paderborn 07 FC Augsburg RB Leipzig
Borussia Dortmund Bayern Munich FC Augsburg Leverkusen
Borussia Dortmund M'gladbach FC Augsburg SC Freiburg
Borussia Dortmund Wolfsburg FC Au

Borussia Dortmund Wolfsburg FC Augsburg Bayern Munich
ooooooooooooooooooooooooooooooooooooooooooooooooo
Bayer Leverkusen Bayern Munich SC Paderborn 07 Hertha
Bayer Leverkusen Dortmund SC Paderborn 07 FC Augsburg
Bayer Leverkusen SC Freiburg SC Paderborn 07 Mainz
Bayer Leverkusen Leverkusen SC Paderborn 07 SC Paderborn 07
Bayer Leverkusen Werder Bremen SC Paderborn 07 Fortuna Dusseldorf
Bayer Leverkusen Wolfsburg SC Paderborn 07 FC Koln
Bayer Leverkusen M'gladbach SC Paderborn 07 Schalke
Bayer Leverkusen Ein Frankfurt SC Paderborn 07 Hoffenheim
Bayer Leverkusen 1. FC Union Berlin SC Paderborn 07 RB Leipzig
Bayer Leverkusen FC Koln SC Paderborn 07 Dortmund
Bayer Leverkusen FC Augsburg SC Paderborn 07 1. FC Union Berlin
Bayer Leverkusen Fortuna Dusseldorf SC Paderborn 07 Leverkusen
Bayer Leverkusen Hoffenheim SC Paderborn 07 Werder Bremen
Bayer Leverkusen Mainz SC Paderborn 07 M'gladbach
Bayer Leverkusen SC Paderborn 07 SC Paderborn 07 SC Freiburg
Bayer Leverkusen Schalke SC Paderborn 07 

Bayer Leverkusen Schalke SC Paderborn 07 SC Freiburg
Bayer Leverkusen Hertha SC Paderborn 07 M'gladbach
Bayer Leverkusen Fortuna Dusseldorf SC Paderborn 07 1. FC Union Berlin
Bayer Leverkusen SC Paderborn 07 SC Paderborn 07 Ein Frankfurt
Bayer Leverkusen Schalke SC Paderborn 07 M'gladbach
Bayer Leverkusen FC Augsburg SC Paderborn 07 Dortmund
Bayer Leverkusen FC Koln SC Paderborn 07 Wolfsburg
Bayer Leverkusen Fortuna Dusseldorf SC Paderborn 07 Werder Bremen
Bayer Leverkusen Hoffenheim SC Paderborn 07 Ein Frankfurt
Bayer Leverkusen Mainz SC Paderborn 07 SC Freiburg
Bayer Leverkusen RB Leipzig SC Paderborn 07 1. FC Union Berlin
Bayer Leverkusen Hertha SC Paderborn 07 Bayern Munich
Bayer Leverkusen SC Paderborn 07 SC Paderborn 07 Leverkusen
Bayer Leverkusen Dortmund SC Paderborn 07 FC Koln
Bayer Leverkusen Ein Frankfurt SC Paderborn 07 RB Leipzig
Bayer Leverkusen SC Freiburg SC Paderborn 07 SC Paderborn 07
Bayer Leverkusen M'gladbach SC Paderborn 07 Mainz
Bayer Leverkusen 1. FC Union Berli

...

Eintracht Frankfurt Mainz Fortuna Dusseldorf Ein Frankfurt
Eintracht Frankfurt Ein Frankfurt Fortuna Dusseldorf Hertha
Eintracht Frankfurt FC Augsburg Fortuna Dusseldorf Mainz
Eintracht Frankfurt Dortmund Fortuna Dusseldorf Fortuna Dusseldorf
Eintracht Frankfurt SC Freiburg Fortuna Dusseldorf Wolfsburg
Eintracht Frankfurt M'gladbach Fortuna Dusseldorf Bayern Munich
Eintracht Frankfurt RB Leipzig Fortuna Dusseldorf Hoffenheim
Eintracht Frankfurt Leverkusen Fortuna Dusseldorf Schalke
Eintracht Frankfurt 1. FC Union Berlin Fortuna Dusseldorf FC Koln
Eintracht Frankfurt Werder Bremen Fortuna Dusseldorf SC Paderborn 07
Eintracht Frankfurt Hoffenheim Fortuna Dusseldorf FC Augsburg
Eintracht Frankfurt Bayern Munich Fortuna Dusseldorf Werder Bremen
Eintracht Frankfurt FC Koln Fortuna Dusseldorf Leverkusen
Eintracht Frankfurt Hertha Fortuna Dusseldorf SC Freiburg
Eintracht Frankfurt Mainz Fortuna Dusseldorf Dortmund
Eintracht Frankfurt SC Paderborn 07 Fortuna Dusseldorf 1. FC Union Berlin
Eintr

Eintracht Frankfurt M'gladbach Fortuna Dusseldorf Hertha
Eintracht Frankfurt 1. FC Union Berlin Fortuna Dusseldorf Fortuna Dusseldorf
Eintracht Frankfurt Werder Bremen Fortuna Dusseldorf FC Koln
Eintracht Frankfurt Wolfsburg Fortuna Dusseldorf Bayern Munich
ooooooooooooooooooooooooooooooooooooooooooooooooo
Fortuna Dusseldorf Bayern Munich Wolfsburg Hertha
Fortuna Dusseldorf Dortmund Wolfsburg FC Augsburg
Fortuna Dusseldorf SC Freiburg Wolfsburg Mainz
Fortuna Dusseldorf Leverkusen Wolfsburg SC Paderborn 07
Fortuna Dusseldorf Werder Bremen Wolfsburg Fortuna Dusseldorf
Fortuna Dusseldorf Wolfsburg Wolfsburg FC Koln
Fortuna Dusseldorf M'gladbach Wolfsburg Schalke
Fortuna Dusseldorf Ein Frankfurt Wolfsburg Hoffenheim
Fortuna Dusseldorf 1. FC Union Berlin Wolfsburg RB Leipzig
Fortuna Dusseldorf FC Koln Wolfsburg Dortmund
Fortuna Dusseldorf FC Augsburg Wolfsburg 1. FC Union Berlin
Fortuna Dusseldorf Fortuna Dusseldorf Wolfsburg Leverkusen
Fortuna Dusseldorf Hoffenheim Wolfsburg Werder Bremen


Mainz Mainz Hertha Berlin SC Freiburg
Mainz RB Leipzig Hertha Berlin 1. FC Union Berlin
Mainz Hertha Hertha Berlin Bayern Munich
Mainz SC Paderborn 07 Hertha Berlin Leverkusen
Mainz Dortmund Hertha Berlin FC Koln
Mainz Ein Frankfurt Hertha Berlin RB Leipzig
Mainz SC Freiburg Hertha Berlin SC Paderborn 07
Mainz M'gladbach Hertha Berlin Mainz
Mainz 1. FC Union Berlin Hertha Berlin FC Augsburg
Mainz Wolfsburg Hertha Berlin Hertha
Mainz Bayern Munich Hertha Berlin Schalke
Mainz Werder Bremen Hertha Berlin Hoffenheim
Mainz Leverkusen Hertha Berlin Fortuna Dusseldorf
Mainz Hertha Hertha Berlin Schalke
Mainz FC Augsburg Hertha Berlin Werder Bremen
Mainz Dortmund Hertha Berlin 1. FC Union Berlin
Mainz Fortuna Dusseldorf Hertha Berlin Ein Frankfurt
Mainz Hoffenheim Hertha Berlin Leverkusen
Mainz Mainz Hertha Berlin Bayern Munich
Mainz RB Leipzig Hertha Berlin M'gladbach
Mainz FC Koln Hertha Berlin SC Freiburg
Mainz SC Paderborn 07 Hertha Berlin Wolfsburg
Mainz Ein Frankfurt Hertha Berlin FC Aug

FC Augsburg Leverkusen Eintracht Frankfurt 1. FC Union Berlin
FC Augsburg Werder Bremen Eintracht Frankfurt RB Leipzig
FC Augsburg M'gladbach Eintracht Frankfurt Fortuna Dusseldorf
FC Augsburg Ein Frankfurt Eintracht Frankfurt Dortmund
FC Augsburg Wolfsburg Eintracht Frankfurt Hoffenheim
FC Augsburg 1. FC Union Berlin Eintracht Frankfurt Ein Frankfurt
FC Augsburg FC Augsburg Eintracht Frankfurt Leverkusen
FC Augsburg

KeyboardInterrupt: 

In [2]:
under=pd.read_csv('Uunder.csv')
#sky1=pd.read_csv('Sky400000.csv')
#sky2=pd.read_csv('skyLeagues385_400k.csv')
#sky3=pd.read_csv('skyLeagues385_400k2.csv')
#sky4=pd.read_csv('C://Users//joaom//Desktop//WScrapy//SkySports//SkyL385-390k.csv')
#sky5=pd.read_csv('C://Users//joaom//Desktop//WScrapy//SkySports//SkyL365_385.csv')
#sky6=pd.read_csv('C://Users//joaom//Desktop//WScrapy//SkySports//Sky325-365k.csv')
#sky7=pd.read_csv('C://Users//joaom//Desktop//WScrapy//SkySports//Sky325-365k2.csv')
#sky8=pd.read_csv('C://Users//joaom//Desktop//WScrapy//SkySports//Sky300-325k.csv')
sky9 = pd.read_csv('Sky365_385.csv')
sky10 = pd.read_csv('Sky385_400.csv')
sky11 = pd.read_csv('Sky400_425.csv')

In [3]:
sky=pd.concat([sky9,sky10,sky11])

In [4]:
sky = sky.dropna(how='all')
sky = sky.drop_duplicates()
sky = sky.reset_index(drop=True)

In [5]:
under = under[:-1]

In [6]:
under = Under_Desktop_drop(under)
sky = Sky_Desktop_drop(sky)

 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está nas leagues
 Southern Premier League não está 

 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Serie B não está nas leagues
 Italian Ser

 UEFA Champions League Group A não está nas leagues
 UEFA Champions League Group C não está nas leagues
 UEFA Champions League Group C não está nas leagues
 UEFA Champions League Group B não está nas leagues
 UEFA Champions League Group D não está nas leagues
 UEFA Champions League Group D não está nas leagues
 UEFA Champions League Group B não está nas leagues
 UEFA Champions League Group A não está nas leagues
 UEFA Champions League Group D não está nas leagues
 UEFA Champions League Group B não está nas leagues
 UEFA Champions League Group B não está nas leagues
 UEFA Champions League Group C não está nas leagues
 UEFA Champions League Group A não está nas leagues
 UEFA Champions League Group A não está nas leagues
 UEFA Champions League Group C não está nas leagues
 UEFA Champions League Group D não está nas leagues
 UEFA Champions League Group F não está nas leagues
 UEFA Champions League Group G não está nas leagues
 UEFA Champions League Group G não está nas leagues
 UEFA Champi

 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas leagues
 FIFA World Cup CONCACAF Qualifying 5th Round não está nas lea

 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagues
 The FA Trophy Third Round Qualifiers não está nas leagu

...

 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The FA Trophy First Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round não está nas leagues
 The Women's FA Cup Second Round n

 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South American  Qualifying Tournament não está nas leagues
 FIFA World Cup South Ame

 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of Ireland First Division não está nas leagues
 League of

 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Chinese Super League não está nas leagues
 Spanish Copa del Rey Fourth Round não está nas league

 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Norwegian Eliteserien não está nas leagues
 Hungarian Liga não está nas leagues
 Swedish Allsvenskan não está nas leagues
 Swedish Allsvenskan não está nas leagues
 Swedish Allsvenskan não está nas leagues
 Swedish Allsvenskan não está nas leagues
 Swedish Allsvenskan não está nas leagues
 Swed

In [18]:
under1=Prepare__Df(under,2020,'Premier_League')
sky1=Prepare__Df(sky,2020,'Premier_League')

Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"
Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"


In [20]:
len(sky1)

380

In [138]:
same = file_teams2(sky,under,'La_Liga')

Não leu ficheiro
------
['Athletic Bilbao']
Athletic Club? (write "passa" if not sure Or "reset" to restart) Athletic Bilbao


In [21]:
%time a,b=Merge_Sky_Under(sky1,under1)

Não leu ficheiro
------
['Leicester City', 'West Ham United', 'Brighton and Hove Albion', 'Tottenham Hotspur', 'Norwich City']
Tottenham? (write "passa" if not sure Or "reset" to restart) Tottenham Hotspur
------
['Leicester City', 'West Ham United', 'Brighton and Hove Albion', 'Norwich City']
Leicester? (write "passa" if not sure Or "reset" to restart) Leicester City
------
['West Ham United', 'Brighton and Hove Albion', 'Norwich City']
Norwich? (write "passa" if not sure Or "reset" to restart) Norwich City
------
['West Ham United', 'Brighton and Hove Albion']
Brighton? (write "passa" if not sure Or "reset" to restart) Brighton and Hove Albion
------
['West Ham United']
West Ham? (write "passa" if not sure Or "reset" to restart) West Ham United
sky 380 under 380
0 of  380
s 0
1 of  380
s 1
2 of  380
s 2
3 of  380
s 3
4 of  380
s 4
5 of  380
s 5
6 of  380
s 6
7 of  380
s 8
8 of  380
s 7
9 of  380
s 9
10 of  380
s 16
11 of  380
s 15
12 of  380
s 14
13 of  380
s 11
14 of  380
s 12
15 of

In [54]:
np.where(np.logical_and(sky1['HT']=='Reims',sky1['AT']=='Metz'))

(array([204], dtype=int64),)

In [51]:
under1.iloc[163]

ADEEP                             5
APPDA                         19.23
AS                               18
AST                               3
AT                             Metz
AxG                            0.94
AxPTS                          0.85
Date            2019-12-07 00:00:00
FTAG                              1
FTHG                              4
HDEEP                             5
HPPDA                          8.66
HS                               14
HST                               5
HT                             Nice
HxG                            1.52
HxPTS                          1.87
League                      Ligue_1
UnderStatsID                  12870
Name: 163, dtype: object

In [25]:
len(a)

306

In [11]:
def save_files(path,w,names): 
    print('Não esquecer do "//" no final path')
    print('w e names têm de estar dentro de uma lista')
    #guarda os ficheiros em w com os nomes em names
    for n, df in enumerate(w):
        df=df.sort_values(['Date','Time'], ascending=[True,True])
        with pd.ExcelWriter(path+names[n]+".xlsx") as writer:
            df.to_excel(writer, names[n], index=None)
        writer.save()

In [22]:
save_files('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//NewPremier_League___//',[a],['NewPL19_20'])

Não esquecer do "//" no final path
w e names têm de estar dentro de uma lista


In [24]:
with pd.ExcelWriter('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//NewSerie_A___'+'NewSA19_20'+".xlsx") as writer:
    a.to_excel(writer, 'NewSA19_20', index=None)
writer.save()

In [19]:
os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//NewSerie_A___')

['NewSA14_15.xlsx',
 'NewSA15_16.xlsx',
 'NewSA16_17.xlsx',
 'NewSA17_18.xlsx',
 'NewSA18_19.xlsx']

In [14]:
a.to_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//NewSerie_A___',index=False)

,League,Date,Time,Attendance,HT,AT,FTR,FTHG,FTAG,HxG,AxG,HxPTS,AxPTS,HP,AP,HS,AS,HST,AST,HSOT,ASOT,HBS,ABS,HCCC,ACCC,HPass,APass,HDEEP,ADEEP,HPPDA,APPDA,HTack,ATack,HAD,AAD,HCorn,ACorn,HOff,AOff,HSaves,ASaves,HF,AF,HY,AY,HR,AR
0,Italian Serie A,2019-08-24,1700,"20,073.",Parma,Juventus,A,0,1,0.26,1.89,0.23,2.63,38.6,61.4,8.0,14.0,2.0,4.0,2.0,7.0,4.0,3.0,0.0,3.0,78.9,88.6,1,9,10.67,6.62,50.0,41.2,42.1,57.9,5.0,8.0,2.0,1.0,3.0,2.0,13.0,13.0,2.0,3.0,0.0,0.0
0,Italian Serie A,2019-08-24,1945,"33,614.",Fiorentina,Napoli,A,3,4,1.94,1.95,1.34,1.38,42.9,57.1,20.0,16.0,6.0,7.0,6.0,3.0,8.0,6.0,0.0,1.0,82.5,85.4,2,4,10.10,7.53,64.3,50.0,42.9,57.1,4.0,6.0,1.0,0.0,3.0,3.0,21.0,21.0,2.0,4.0,0.0,0.0
0,Italian Serie A,2019-08-25,1700,"24,584.",Udinese,AC Milan,H,1,0,0.78,0.63,1.43,1.19,40.7,59.3,17.0,14.0,6.0,0.0,6.0,9.0,5.0,5.0,0.0,0.0,82.2,88.2,2,3,19.44,10.00,58.3,42.9,48.3,51.7,9.0,4.0,0.0,0.0,0.0,5.0,16.0,16.0,1.0,3.0,0.0,0.0
0,Italian Serie A,2019-08-25,1945,"16,324.",Verona,Bologna,D,1,1,0.11,2.11,0.09,2.83,25.6,74.4,5.0,16.0,2.0,2.0,3.0,9.0,0.0,5.0,0.0,2.0,56.2,87.0,0,14,15.94,4.47,56.3,73.3,56.4,43.6,1.0,6.0,2.0,1.0,1.0,1.0,13.0,13.0,1.0,3.0,1.0,0.0
0,Italian Serie A,2019-08-25,1945,"38,779.",Roma,Genoa,D,3,3,2.63,1.13,2.41,0.42,53.6,46.4,23.0,6.0,9.0,4.0,10.0,2.0,4.0,0.0,3.0,1.0,85.1,83.1,16,6,18.21,7.83,90.0,38.5,44.4,55.6,5.0,1.0,0.0,6.0,1.0,6.0,14.0,14.0,3.0,1.0,0.0,0.0


In [3]:
add_odds('Ligue_1')

6
iiiiiiiiiii 0
C:\Users\joaom\Documents\Projetos\PYTHON\Apostas\Ligue_1___\L14_15 old
C:\Users\joaom\Documents\Projetos\PYTHON\Apostas\NewLigue_1___\NewL14_15 new
1o arg é o NewFile
['Reims', 'Guingamp', 'Lille', 'Montpellier', 'Nantes', 'Nice', 'Bastia', 'Lyon', 'Monaco', 'Caen', 'Paris Saint-Germain', 'Toulouse', 'RC Lens', 'Lorient', 'Metz', 'Marseille', 'St Etienne', 'Bordeaux', 'Rennes', 'Evian Thonon Gaillar']
--------
['Reims', 'Bastia', 'Evian Thonon Gaillard', 'Guingamp', 'Lille', 'Montpellier', 'Nantes', 'Nice', 'Lyon', 'Monaco', 'Caen', 'Lens', 'Lorient', 'Metz', 'Paris SG', 'Rennes', 'Toulouse', 'Bordeaux', 'Marseille', 'St Etienne']
------
['Paris Saint-Germain', 'RC Lens', 'Evian Thonon Gaillar']
Evian Thonon Gaillard? (write "passa" if not sure Or "reset" to restart) Evian Thonon Gaillar
------
['Paris Saint-Germain', 'RC Lens']
Lens? (write "passa" if not sure Or "reset" to restart) RC Lens
------
['Paris Saint-Germain']
Paris SG? (write "passa" if not sure Or "reset" 

In [134]:
def file_teams2(w1,w2,league): #w1 -> df Sky |||| w2 -> df Under
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('Teams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome Under]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        under_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
        under_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
    i = 0
    under_teams_copy = under_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(under_teams) and not(found):
                if sky_teams[ii]==under_teams[nn]:
                    same+=[[sky_teams[ii],under_teams[nn]]]
                    del under_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            under_teams_copy=under_teams.copy()
            same_copy=same.copy()
            t=0
            while under_teams!=[]:
                print('------')
                print(restos)
                ok=input(str(under_teams[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    under_teams=under_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,under_teams[t]]]
                    del under_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(under_teams):
                    t=0
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_under = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_under = [under_teams[i] for i in range(len(under_teams)) if (under_teams[i] not in same_under)]
        missing_sky_copy = missing_sky.copy()
        missing_under_copy = missing_under.copy()
        same_copy=same.copy()
        
        restos=[]
        for ii in range(len(missing_sky)):
            nn=0
            found=False
            while nn<len(missing_under) and not(found):
                if missing_sky[ii]==missing_under[nn]:
                    same+=[[missing_sky[ii],missing_under[nn]]]
                    del missing_under[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[missing_sky[ii]]

        if restos==[]:
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_under!=[]):
                if len(restos) != len(missing_under):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input(str(missing_under[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_under=missing_under_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_under[t]]]
                    del missing_under[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_under):
                    t=0
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        

In [141]:
def update_team_names(sky,under,league):
# vamos mudar o nome das dfs de under para o nome das dfs do sky
#assim podemos fazer um merge usando o np.where
    same = file_teams2(sky,under,league)
    same_sky = [same[i][0] for i in range(len(same))]
    same_under = [same[i][1] for i in range(len(same))]
    #HOME
    for i in range(len(same_under)):
        
        try:
            replaceH=list(np.where(under['HT']==same_under[i])[0])
            home='HT'
        except:
            replaceH=list(np.where(under['HomeTeam']==same_under[i])[0])
            home='HomeTeam'
        
        for n in replaceH:
            under.at[n,home]=same_sky[i]
    #AWAY
    for i in range(len(same_under)):
        try:
            replaceH=list(np.where(under['AT']==same_under[i])[0])
            away='AT'
        except:
            replaceH=list(np.where(under['AwayTeam']==same_under[i])[0])
            away='AwayTeam'
        for n in replaceH:
            under.at[n,away]=same_sky[i]  
    return sky,under

In [142]:
sky1,under1=update_team_names(sky,under,'La_Liga')

In [144]:
a=200
np.where(np.logical_and(sky['HT']==under.iloc[a]['HT'],sky['AT']==under.iloc[a]['AT']))

(array([199], dtype=int64),)

In [174]:
def Merge_Sky_Under(sky,under):
    leagues={
        'EPL':'Premier_League','Bundesliga':'Bundesliga','Ligue 1':'Ligue_1',
        'RFPL':'RFPL','La liga':'La_Liga','Serie A':'Serie_A'}
    if under.iloc[0]['League'] in leagues:
        league = leagues[under.iloc[0]['League']]
    else:
        print('Falta a Liga: '+str(under.iloc[0]['League']))
        raise('aa')
    sky,under=update_team_names(sky,under,league)
    sky_size=len(sky.index)
    under_size=len(under.index)
    print('sky',sky_size,'under',under_size)
    #sky_range=list(range(sky_size)) #esta lista vai dar jeito para tornar o programa mais rápido
    no_find=[] #lista dos jogos não encontrados
    pre_df=[]
    dix=file_teams(sky,under) #lista com as equipas iguais para o check_game_same
    for u in range(under_size):
        print(u, 'of ',under_size)
        
        sg = list(np.where(np.logical_and(sky['HT']==under.iloc[u]['HT'],sky['AT']==under.iloc[u]['AT']))[0])
        if len(sg)>1:
            print('Whaaaatt?, double game')
            raise('aa')
        if sg!=[]:
            s=sg[0]
            if is_there_nan(sky,s):
                continue
            else:
                #print('2')
                datassa=SkySports_Date(sky.iloc[s]['Date'])
                if type(sky.iloc[s]['Date'])==pd._libs.tslibs.timestamps.Timestamp:
                    data__=sky.iloc[s]['Date']
                    time__=sky.iloc[s]['Time']
                else:
                    data__=datassa[0]
                    time__=datassa[1]
                print('s',s)
                pre_df+=[[
                    clear_(sky.iloc[s]['League']), data__, 
                    time__, sky.iloc[s]['Attendance'],
                    sky.iloc[s]['HT'], sky.iloc[s]['AT'],
                    ['H' if under.iloc[u]['FTHG']>under.iloc[u]['FTAG'] else 'D' if under.iloc[u]['FTHG']==under.iloc[u]['FTAG'] else 'A'][0] ,
                    under.iloc[u]['FTHG'], 
                    under.iloc[u]['FTAG'], under.iloc[u]['HxG'], under.iloc[u]['AxG'], 
                    under.iloc[u]['HxPTS'], under.iloc[u]['AxPTS'], 
                    sky.iloc[s]['HP'], sky.iloc[s]['AP'], 
                    sky.iloc[s]['HS'], sky.iloc[s]['AS'], sky.iloc[s]['HST'], 
                    sky.iloc[s]['AST'], sky.iloc[s]['HSOT'], sky.iloc[s]['ASOT'], 
                    sky.iloc[s]['HBS'], sky.iloc[s]['ABS'],
                    sky.iloc[s]['HCCC'], sky.iloc[s]['ACCC'], sky.iloc[s]['HPass'], 
                    sky.iloc[s]['APass'], under.iloc[u]['HDEEP'], under.iloc[u]['ADEEP'], 
                    under.iloc[u]['HPPDA'], under.iloc[u]['APPDA'], 
                    sky.iloc[s]['HTack'], sky.iloc[s]['ATack'], 
                    sky.iloc[s]['HAD'], sky.iloc[s]['AAD'],
                    sky.iloc[s]['HCorn'], sky.iloc[s]['ACorn'], sky.iloc[s]['HOff'], 
                    sky.iloc[s]['AOff'], sky.iloc[s]['HSaves'], sky.iloc[s]['ASaves'], 
                    sky.iloc[s]['HF'], sky.iloc[s]['AF'], sky.iloc[s]['HY'], 
                    sky.iloc[s]['AY'], sky.iloc[s]['HR'], sky.iloc[s]['AR']
                ]]
        else:
            no_find+=[under.iloc[u]['UnderStatsID']]
    qw=[] 
    print('last here we go')
    for nn in range(len(pre_df)):
        #print('nn',nn)
        dta=pd.DataFrame({
            'League':pre_df[nn][0],'Date':pre_df[nn][1],'Time':pre_df[nn][2],'Attendance':pre_df[nn][3],
            'HT':pre_df[nn][4],'AT':pre_df[nn][5],'FTR':pre_df[nn][6],'FTHG':pre_df[nn][7],'FTAG':pre_df[nn][8],
            'HxG':pre_df[nn][9],'AxG':pre_df[nn][10],'HxPTS':pre_df[nn][11],'AxPTS':pre_df[nn][12],
            'HP':pre_df[nn][13],'AP':pre_df[nn][14],'HS':pre_df[nn][15],'AS':pre_df[nn][16],
            'HST':pre_df[nn][17],'AST':pre_df[nn][18],'HSOT':pre_df[nn][19],'ASOT':pre_df[nn][20],
            'HBS':pre_df[nn][21],'ABS':pre_df[nn][22],'HCCC':pre_df[nn][23],'ACCC':pre_df[nn][24],
            'HPass':pre_df[nn][25],'APass':pre_df[nn][26],'HDEEP':pre_df[nn][27],'ADEEP':pre_df[nn][28],
            'HPPDA':pre_df[nn][29],'APPDA':pre_df[nn][30],'HTack':pre_df[nn][31],'ATack':pre_df[nn][32],
            'HAD':pre_df[nn][33],'AAD':pre_df[nn][34],'HCorn':pre_df[nn][35],'ACorn':pre_df[nn][36],
            'HOff':pre_df[nn][37],'AOff':pre_df[nn][38],'HSaves':pre_df[nn][39],'ASaves':pre_df[nn][40],
            'HF':pre_df[nn][41],'AF':pre_df[nn][42],'HY':pre_df[nn][43],'AY':pre_df[nn][44],
            'HR':pre_df[nn][45],'AR':pre_df[nn][46]
        },index=[0])
        qw+=[dta]
        #print('aha',len(qw))
    one=pd.concat(qw,sort=False)
    print('original_size: sky',sky_size,'| under',under_size)
    print('no_find',no_find)
    print('file_size',len(one.index))
    return one, no_find